# Preeliminaries

### Rpy2 package

In [1]:
%load_ext rpy2.ipython

### Importing R packages

In [2]:
%R library(glmnet)
%R library(dplyr)
%R library(rvinecopulib)
%R library(knockoff)
#%R library(performanceEstimation)
%R library(foreach)
%R library(doParallel)
%R library(TSP)
%R library(VineCopula)
#%R library(ROSE)

R[write to console]: Loading required package: Matrix

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: Loading required package: iterators

R[write to console]: Loading required package: parallel



'VineCopu...,'TSP','doParall...,...,'datasets','methods','base'


### Importing Python libraries

In [3]:
import pandas as pd
import warnings
import numpy as np
from timeit import default_timer as timer
import math


#For parallel computing
import multiprocessing
from joblib import Parallel, delayed
#Number of cores
num_cores = multiprocessing.cpu_count()
jobs=num_cores-1


import matplotlib.pyplot as plt
%matplotlib inline

#knockpy (knockoffs)
import knockpy
from knockpy.knockoff_filter import KnockoffFilter
#from knockpy.knockoff_stats import data_dependent_threshhold


#vineknockoffs
from vineknockoffs.vine_knockoffs import VineKnockoffs


In [4]:
#Import the package rpy2
import rpy2
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from rpy2.robjects.conversion import localconverter

# import R's packages
base = importr('base')
glmnet = importr('glmnet')
dplyr = importr('dplyr')


#### Python functions

In [5]:
#Function to make selections
def make_selections(W, fdr):
    """" Calculate data dependent threshhold and selections
    returns a np.ndarray
    
    Parameters 
    ---------- 
    W : np.ndarray 
    fdr : float
    """  
    
    threshold = data_dependent_threshhold(W=W, fdr=fdr)
    selected_flags = (W >= threshold).astype("float32")
    return selected_flags


def lasso_glmnet_lambda_min(x):
  """" Find the tuning lambda using the R package glmnet 
  
  Returns a robjects.vectors.FloatVector
  
  Parameters 
  ---------- 
  x : pandas.DataFrame  
  """    
    
  #Convertion of the pandas dataframe to a R dataframe  
  sim = x
  with localconverter(robjects.default_converter + pandas2ri.converter):
    r_sim = robjects.conversion.py2rpy(sim)
  robjects.globalenv["r_sim"] = r_sim
  
  #Loading R libraries  
  base = importr('base')
  glmnet = importr('glmnet')
  dplyr = importr('dplyr')


  #Fitting the Cox’s proportional hazards model employing glmnet
  robjects.r(''' 
        X <- r_sim %>% select(-c("Class"))
        X_matrix <- as.matrix(X)
        y <- r_sim %>% select(c("Class"))
        y <- as.matrix(y)
        cvfit <- cv.glmnet(X_matrix, y, alpha=1, family = "binomial",  nfolds = 10, standardize = TRUE)
        lambda_min_r <- as.numeric(cvfit$lambda.min)
        ''')
  #Tuning lambda
  lambda_min = robjects.globalenv['lambda_min_r']  
  
  return lambda_min  


In [6]:
def vineknockoffs_estimation(X):
     
    vine_ko = VineKnockoffs()
    vine_ko.fit_vine_copula_knockoffs(x_train=X)      
    
    return vine_ko


def vineknockoffs_generation(X, vine_ko, m):
    
    np.random.seed(m) 
    try:
        Xk_vineknockoffs = vine_ko.generate(X)      
    except:
        Xk_vineknockoffs = None
    
    return Xk_vineknockoffs

### R functions related to dvines

In [7]:
%%R

#X_Xk_dvine_distributions()-->Function to fit the dvine distribution for X and X_X matrices
#Arguments:
#X: matrix of predictors
#vinecop_family : String related to the family of pair copulas used in the dvine fitting. 
#                 Common options are "parametric", "nonparametric", "onepar". More details can be found
#                 in the documentation of R package rvinecopulib
#                  https://cran.r-project.org/web/packages/rvinecopulib/rvinecopulib.pdf
# n_cores: int -> number of cores for parallel processing

#Value: This function returns a list that contains objects of class vinecop_dist for X and X_X
#Note: more information about objects of class vinecop_dist can be found in 
#https://cran.r-project.org/web/packages/rvinecopulib/rvinecopulib.pdf

X_Xk_dvine_distributions <- function(X, vinecop_family="parametric", n_cores=1){
    
    if (is.null(X)) {
        stop("Argument X is null")  
    }
    
    #Number of variables p and sample size n
    n <- dim(X)[1]
    p <- dim(X)[2]


    #dstructures for dvines
    X_X_dstructure <- dvine_structure((2*p):1)
    X_dstructure <- dvine_structure(p:1)

    #Dataset column binding
    X_X <- cbind(X,X)

    #Seudo-Observations
    u_X_X <- pseudo_obs(X_X)

    #Fitting dvine distribution for X_X
    dvine_fitting_time <- system.time(
    fit_dvine_trunc <- vinecop(u_X_X, family_set=c(vinecop_family), structure= X_X_dstructure, presel=TRUE,
                         selcrit='mbicv', par_method='mle', psi0=0.95, show_trace=FALSE, cores=n_cores, trunc_lvl=p-1)
    )

    #Printing dvine X_X fitting time
    print("dvine fitting time in seconds:")
    print(dvine_fitting_time)

    #Pair-copula list for X_X
    X_X_dvine_pclist <- fit_dvine_trunc$pair_copulas

    #dvine distribution for X_X 
    X_X_dvine_dist <- vinecop_dist(X_X_dvine_pclist, X_X_dstructure)

    #Pair-copula list for X
    X_dvine_pclist <- list(rep(list(""),p-1))

    #Iniziating with independent copula
    for (i in 1:(p-1)){
    bicop <- bicop_dist("indep",)
    X_dvine_pclist[i] <- list(rep(list(bicop),p-i))
    }

    #Pair copula list just for X dependencies
    for (i in 1:(p-1)){
    J <- p-i

    for (j in 1:J){
      X_dvine_pclist[[i]][j] <- X_X_dvine_pclist[[i]][j] 

    } 
    }

    # dvine distribution for X
    X_dvine_dist <- vinecop_dist(X_dvine_pclist, X_dstructure)

    #List with dvine distributions
    dvine_distributions <- list(X_dvine_dist=X_dvine_dist, X_X_dvine_dist=X_X_dvine_dist)

    return(dvine_distributions)
}

# create_dvine_Knockoffs()--> Function to sample dvine knockoffs
#Arguments:
#X: matrix of predictors
#X_dvine_dist: Object of class vinecop_dist for X, contaning a list specifying the pair-copulas,
#              structure, and variable types.
#X_X_dvine_dist: Object of class vinecop_dist for X_X, 
#               contaning a list specifying the pair-copulas, structure, and variable types.
#n_cores: int -> number of cores for parallel processing
#Note: more information about objects of class vinecop_dist can be found in 
#https://cran.r-project.org/web/packages/rvinecopulib/rvinecopulib.pdf

#Value: This function returns a matrix Xk of knockoffs

create_dvine_Knockoffs <- function(X, X_dvine_dist , X_X_dvine_dist, n_cores=1){

    if (is.null(X)) {
        stop("Argument X is null")  
    }
    if (is.null( X_dvine_dist)) {
        stop("Argument X_dvine_dist is null")  
    }
    if (is.null( X_X_dvine_dist)) {
        stop("Argument X_X_dvine_dist is null")  
    }
           
    #Number of variables p and sample size n
    n <- dim(X)[1]
    p <- dim(X)[2]
        
    #Pseudo observations
    u_X <- pseudo_obs(X)

    #Independent uniforms w
    w_X <- rosenblatt(x=u_X, model=X_dvine_dist, cores = n_cores)
    w_Xk <- matrix(runif(n=p*n,min=0,max=1),nrow=n,ncol=p)
    w_X_Xk <- cbind(w_X,w_Xk)

    #Knockoff sampling Xk
    u_X_Xk <- inverse_rosenblatt(u=w_X_Xk, model= X_X_dvine_dist, cores = n_cores)
    u_Xk <- u_X_Xk[,(p+1):(2*p)]

    #Marginal transformation
    Xk <- X
    for(i in 1:p) {   
        Xk[,i] <- as.vector(quantile(X[,i], probs=punif(u_Xk[,i],min=0, max=1), type=8))
    }

    return(Xk)
}

# stable_lasso_glmnet()--> Function to fit a regularized lasso regresion model using 
#some functions of the R package glmnet.
#It implements the stabilizing procedure of Roberts and Nowak (2014) to diminish sensitivity
#to the fold assignment used in cross-validation to select the hyperparameter lambda

#Arguments:
#X: matrix of predictors
#y: vector or matrix of response
#lasso_family: a string to select linear regression "gaussian" or logistic regression "binomial"
#M_lasso: integer related to the number of runs for the stabilzation against CV
#n_folds: integer indicating the number of cross validations
#Note: more information about the R package glmnet can be found in 
#https://cran.r-project.org/web/packages/glmnet/glmnet.pdf
#Note 2: this function runs in parallel for the stabilzation against CV

#Value: This function returns a vector of the estimated coeficientes (without the intercept)

stable_lasso_glmnet <- function(X, y, lasso_family, M_lasso = 10, n_folds = 5){
    
    if (is.null(X)) {
        stop("Argument X is missing")  
    }
    if (is.null(y)) {
        stop("Argument y is missing")  
    }
    if (is.null( lasso_family )) {
        stop("Argument lasso_family is missing")  
    }

    
    y_vec <- as.vector(y)
    X_matrix <- as.matrix(X)
    
    #Stabilizing the lasso against CV (Roberts and Nowak, 2014)
    lambdas <- rep(0,M_lasso)

    time_cv <- system.time(  
        lambdas <- foreach(i = 1:M_lasso, .combine=c,.packages=c("glmnet")) %dopar% {
        set.seed(i) #The seed is set for reproducibility proposes
        cvfit <- cv.glmnet(X_matrix, y_vec, alpha=1, family = lasso_family, nfolds = n_folds, standardize = TRUE)
        cvfit$lambda.min
        }
    )

    #Selecting the median of the lambdas distribution
    lambda50 <- as.numeric(quantile(lambdas,probs=0.5))
    fit_coef <- coef(glmnet(X_matrix, y_vec, alpha = 1, lambda = lambda50, family = lasso_family, standardize = TRUE))

    fit_coef_vec <- as.vector(fit_coef)
    fit_coef_vec <- fit_coef_vec[-1] 

    return(fit_coef_vec)
}

# ekn_dvines()--> Function to derandomized knockoffs using e-values for FDR control. This function
# considers the dvine knockoff procedure.
#The code to implement this function is adapted from 
#https://github.com/zhimeir/derandomized_knockoffs_fdr

#Arguments:
#X: matrix of predictors
#y: vector or matrix of response
#M: integer denoting the number of generated copies of the knockff matrix Xk.
#dvine_distributions: list that contains objects of class vinecop_dist for X and X_X
#M_lasso: integer related to the number of runs for the stabilzation against CV
#alpha: integer indicating FDR target level
#gamma: integer denoting target level for the knockoff threshold. According to Ren & Barber (2023),
#       experimentally, gamma=alpha/2 works well.           
#lasso_family: a string to select linear regression "gaussian" or logistic regression "binomial" 
#n_cores: int -> number of cores for parallel processing

#Note: the knockoff.threshold() function from the R knockoff package is used for 
#setting the Knockoff rejection threshold (https://cran.r-project.org/web/packages/knockoff/knockoff.pdf)

#Value: This function returns a list with the selected variables of the procedure

ekn_dvines <- function(X, y, dvine_distributions, M=50, M_lasso=10, alpha=0.2, gamma=0.1, lasso_family, n_cores=1){

    if (is.null(X)) {
        stop("Argument X is missing")  
    }
    if (is.null(y)) {
        stop("Argument y is missing")  
    }
    if (is.null( dvine_distributions )) {
        stop("Argument dvine_distributions is missing")  
    }
    if (is.null( lasso_family )) {
        stop("Argument lasso_family is missing")  
    }
    
    
            
    #Number of variables p and sample size n  
    n <- dim(X)[1]
    p <- dim(X)[2]

    #Initial matrix of E-values 
    E <- matrix(0, M, p)
    
    for(m in 1:M){
        
        set.seed(m) #The seed is set for reproducibility proposes
        
        #dvine Knockoffs sampling
        Xk <- create_dvine_Knockoffs(X, X_dvine_dist = dvine_distributions$X_dvine_dist ,
                                     X_X_dvine_dist =dvine_distributions$X_X_dvine_dist, n_cores)


        #X and Xk column binding
        X_Xk <- cbind(X, Xk)
        
        #Estimated lasso coefficients for X_Xk
        Z <- stable_lasso_glmnet(X_Xk, y, lasso_family, M_lasso)
              
        #Importance statistics
        W <- abs(Z[1:p]) - abs(Z[(p+1):length(Z)])
        
        #Knockoff rejection threshold - conservative procedure ("knockoffs+" offset = 1)
        tau <- stop_early(W, gamma, offset=1) 
        
        #E-vales for all the variables (columns) for m run
        E[m,] <- (W >= tau) / (1 + sum(W <= -tau))
        
   }
    
    #Averaging the e-values to select set of discoveries
    E <- p*colMeans(E)
    rej <- ebh(E, alpha)$rej
    
    return(list(rej = rej, E = E)) 

}

### R functions  to derandomized knockoffs using e-values for FDR control (Gaussian and second order knockoffs)

In [8]:
%%R

ekn_gaussian <- function(X, y, ls_Xk_norm ,M, M_lasso, alpha, gamma, lasso_family, n_cores){

    if (is.null(X)) {
        stop("Argument X is missing")  
    }
    if (is.null(y)) {
        stop("Argument y is missing")  
    }
    if (is.null( lasso_family )) {
        stop("Argument lasso_family is missing")  
    }
   
    #Number of variables p and sample size n  
    n <- dim(X)[1]
    p <- dim(X)[2]

    #Matrix of E-values 
    E <- matrix(0, M, p)
    
    for(m in 1:M){
        
        set.seed(m) #The seed is set for reproducibility proposes
        
        #Gaussian Knockoffs copy selection from the list object
        Xk <- ls_Xk_norm[[m]]
        
        #X and Xk column binding
        X_Xk <- cbind(X, Xk)
        
        #Estimated lasso coefficients for X_Xk
        Z <- stable_lasso_glmnet(X_Xk, y, lasso_family, M_lasso)
        
        #Importance statistics
        W <- abs(Z[1:p]) - abs(Z[(p+1):length(Z)])
                
        #Knockoff rejection threshold - conservative procedure ("knockoffs+" offset = 1)
        tau <- stop_early(W, gamma, offset=1) 
        
        #E-vales for all the variables (columns) for m run
        E[m,] <- (W >= tau) / (1 + sum(W <= -tau))
        
   }
    
    #Averaging the e-values to select set of discoveries
    E <- p*colMeans(E)
    rej <- ebh(E, alpha)$rej
    
    return(list(rej = rej, E = E)) 

}

ekn_vineknockoffs <- function(X, y, ls_Xk_vineknockoffs, M, M_lasso, alpha, gamma, lasso_family, n_cores){

    if (is.null(X)) {
        stop("Argument X is missing")  
    }
    if (is.null(y)) {
        stop("Argument y is missing")  
    }
    if (is.null( lasso_family )) {
        stop("Argument lasso_family is missing")  
    }
    
    
    #Number of variables p and sample size n  
    n <- dim(X)[1]
    p <- dim(X)[2]

    #Matrix of E-values 
    E <- matrix(0, M, p)

    for(m in 1:M){
        
        set.seed(m) #The seed is adjusted for reproducibility issues in the simulations.
        
        #Gaussian Knockoffs copy selection from the list object
        Xk <- ls_Xk_vineknockoffs[[m]]
        
        #X and Xk column binding
        X_Xk <- cbind(X, Xk)
        
        #Estimated lasso coefficients for X_Xk
        Z <- stable_lasso_glmnet(X_Xk, y, lasso_family, M_lasso)
        
        #Importance statistics
        W <- abs(Z[1:p]) - abs(Z[(p+1):length(Z)])
                
        #Knockoff rejection threshold - conservative procedure ("knockoffs+" offset = 1)
        tau <- stop_early(W, gamma, offset=1) 
        
        #E-vales for all the variables (columns) for m run
        E[m,] <- (W >= tau) / (1 + sum(W <= -tau))
        
   }
    
    #Averaging the e-values to select set of discoveries
    E <- p*colMeans(E)
    rej <- ebh(E, alpha)$rej
    
    return(list(rej = rej, E = E)) 

}


ekn_second_order <- function(X, y, M, M_lasso, alpha, gamma, lasso_family, n_cores){

    if (is.null(X)) {
        stop("Argument X is missing")  
    }
    if (is.null(y)) {
        stop("Argument y is missing")  
    }
    if (is.null( lasso_family )) {
        stop("Argument lasso_family is missing")  
    }

    
    #Number of variables p and sample size n  
    n <- dim(X)[1]
    p <- dim(X)[2]

    #Matrix of E-values 
    E <- matrix(0, M, p)

    for(m in 1:M){
        
        set.seed(m) #The seed is set for reproducibility proposes
        
        Xk <- create.second_order(X)
        
        #X and Xk column binding
        X_Xk <- cbind(X, Xk)
        
        #Estimated lasso coefficients for X_Xk
        Z <- stable_lasso_glmnet(X_Xk, y, lasso_family, M_lasso)
        
        #Importance statistics
        W <- abs(Z[1:p]) - abs(Z[(p+1):length(Z)])
 
        #Knockoff rejection threshold - conservative procedure ("knockoffs+" offset = 1)
        tau <- stop_early(W, gamma, offset=1) 
        
        #E-vales for all the variables (columns) for m run
        E[m,] <- (W >= tau) / (1 + sum(W <= -tau))
        
   }
    
    #Averaging the e-values to select set of discoveries
    E <- p*colMeans(E)
    rej <- ebh(E, alpha)$rej
    
    return(list(rej = rej, E = E)) 

}


### Utility functions for the e-values procedure

In [9]:
%%R

#These functions are taken from
#https://github.com/zhimeir/derandomized_knockoffs_fdr


#####################################
## The eBH procedure
#####################################
### Input: 
###   E: e-values
###   alpha: target FDR level
### Output:
###   Variables selected by the e-BH procedure

ebh <- function(E, alpha){
  
  p <- length(E)
  E_ord <- order(E, decreasing = TRUE)
  E <- sort(E, decreasing = TRUE)
  comp <- E >= (p / alpha / (1:p))
  id <- suppressWarnings(max(which(comp>0)))
  if(id > 0){
    rej <- E_ord[1:id]
  }else{
    rej <- NULL
  }
  return(list(rej = rej))
}

#######################################
## Computing the early stopping time ##
#######################################
### Input:
###   W: vector of knockoff feature importance statistics 
###   gamma: alpha_kn 
###   offset: value between 0 and 1
### Output: 
###   The modified knockoff stopping time defined in (14)

stop_early <- function(W, gamma, offset){
  
  tau <- alphakn_threshold(W, fdr =  gamma, offset = offset) 
  ord_W <- order(abs(W), decreasing = TRUE)
  sorted_W <- W[ord_W]
  
  if(sum(W>0) >= 1 / gamma){
    pos_ind <- which(sorted_W > 0)
    tau1 <- sorted_W[pos_ind[ceiling(1/gamma)-1]]
  }else{
    tau1 <- 0
  }
  tau <- min(tau,tau1) 

  return(tau)
}


#######################################################
## Compute stopping time w/ diff alpha_kn and offset ##
#######################################################
### Input:
###   W: a length p vector of knockoff feature importance statistics
###   fdr: the target FDR level
###   offset: 0 or 1 
### Output: 
###   the knockoff selection threshold

alphakn_threshold <- function(W, fdr, offset) {
  ts = sort(c(0, abs(W)))
  ratio = sapply(ts, function(t)
    (offset + sum(W <= -t)) / max(1, sum(W >= t)))
  ok = which(ratio <= fdr)
  ifelse(length(ok) > 0, ts[ok[1]], Inf)
}
                 
                 

###  Stable SMOTE class ( Synthetic Minority Oversampling TEchnique)  

In [10]:
from sklearn import neighbors
from collections import Counter


target_defect_ratio = 0.5 #For balancing the clases 50%-50%

class stable_SMOTE:
    
    def __init__(self, z_nearest=5):
        self.z_nearest = z_nearest
    
    def fit_sample(self, x_dataset):
        x_dataset = pd.DataFrame(x_dataset)      
        
        total_pair = []
        
        #print(k_nearest)
        defective_instance = x_dataset[x_dataset["Class"] == 2] #Minority class (Class==2)
        clean_instance = x_dataset[x_dataset["Class"] == 1] #Mayorito class (Class==1)
        
        defective_number = len(defective_instance)
        clean_number = len(clean_instance)
        need_number = int((target_defect_ratio * len(x_dataset) - defective_number) / (1 - target_defect_ratio))  # clean_number - defective_number
        
        print("Number of minority instances needed")
        print(need_number)
      
        #exit()
        
        if need_number <= 0:
            return False
        
        generated_dataset = []
        synthetic_dataset = pd.DataFrame()
        number_on_each_instance = need_number / defective_number  
        total_pair = []

        rround = number_on_each_instance / self.z_nearest
        
        while rround >= 1:
            for index, row in defective_instance.iterrows():
                temp_defective_instance = defective_instance.copy(deep=True)
                subtraction = row - temp_defective_instance
                square = subtraction ** 2
                row_sum = square.apply(lambda s: s.sum(), axis=1)
                distance = row_sum ** 0.5
                temp_defective_instance["distance"] = distance
                temp_defective_instance = temp_defective_instance.sort_values(by="distance", ascending=True)
                neighbors = temp_defective_instance[1:self.z_nearest + 1]
                for a, r in neighbors.iterrows():
                    selected_pair = [index, a]
                    selected_pair.sort()
                    total_pair.append(selected_pair)
            rround = rround - 1
        need_number1 = need_number - len(total_pair)
        number_on_each_instance = need_number1 / defective_number

        for index, row in defective_instance.iterrows():
            temp_defective_instance = defective_instance.copy(deep=True)
            subtraction = row - temp_defective_instance
            square = subtraction ** 2
            row_sum = square.apply(lambda s: s.sum(), axis=1)
            distance = row_sum ** 0.5
            temp_defective_instance["distance"] = distance
            temp_defective_instance = temp_defective_instance.sort_values(by="distance", ascending=True)
            neighbors = temp_defective_instance[1:self.z_nearest + 1]
            neighbors = neighbors.sort_values(by="distance", ascending=False)  
            target_sample_instance = neighbors[0: int(number_on_each_instance)]
            target_sample_instance = target_sample_instance.drop(columns="distance")
            
            for a, r in target_sample_instance.iterrows():
                selected_pair = [index, a]
                selected_pair.sort()
                total_pair.append(selected_pair)
        
        temp_defective_instance = defective_instance.copy(deep=True)
        residue_number = need_number - len(total_pair)
        residue_defective_instance = temp_defective_instance.sample(n=residue_number)
        
        for index, row in residue_defective_instance.iterrows():
            temp_defective_instance = defective_instance.copy(deep=True)
            subtraction = row - temp_defective_instance
            square = subtraction ** 2
            row_sum = square.apply(lambda s: s.sum(), axis=1)
            distance = row_sum ** 0.5
            temp_defective_instance["distance"] = distance
            temp_defective_instance = temp_defective_instance.sort_values(by="distance", ascending=True)
            neighbors = temp_defective_instance[1:self.z_nearest + 1]
            target_sample_instance = neighbors[-1:]
            
            for a in target_sample_instance.index:
                selected_pair = [index, a]
                selected_pair.sort()
                total_pair.append(selected_pair)
       
        total_pair_tuple = [tuple(l) for l in total_pair]
        result = Counter(total_pair_tuple)
        result_number = len(result)
        result_keys = result.keys()
        result_values = result.values()
        
        for f in range(result_number):
            current_pair = list(result_keys)[f]
            row1_index = current_pair[0]
            row2_index = current_pair[1]
            row1 = defective_instance.loc[row1_index]
            row2 = defective_instance.loc[row2_index]
            generated_num = list(result_values)[f]
            generated_instances = np.linspace(row1, row2, generated_num + 2)
            generated_instances = generated_instances[1:-1]
            generated_instances = generated_instances.tolist()
            for w in generated_instances:
                generated_dataset.append(w)
        final_generated_dataset = pd.DataFrame(data=generated_dataset, columns=clean_instance.columns)
        final_generated_dataset = final_generated_dataset.astype(clean_instance.dtypes.to_dict())
        
        result = pd.concat([clean_instance, defective_instance, final_generated_dataset], axis=0)
        result.reset_index(drop=True, inplace=True)
        return result


### Parameter configuration

In [11]:
#Initial time
ti = timer()

M = 50 #Number of runs for e-values procedure
M_lasso = 100 #Number of runs for Lasso Stability against CV
lasso_family = 'binomial'
vinecop_family = 'parametric'
nonparametric_family = 'nonparametric'
n_cores = 23

#From Python to R

%R -i M
%R -i M_lasso
%R -i lasso_family
%R -i vinecop_family
%R -i nonparametric_family
%R -i n_cores


#Parallel processing
%R registerDoParallel(makeCluster(n_cores))

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x7f7a74bd5b00> [RTYPES.CLOSXP]
R classes: ('function',)

# Loading the gene expression dataset

In [12]:
#Importing the csv file (576 rows and 1001 columns)
Lung_data_complete = pd.read_csv("TGCA_lung_data_1000.csv")
print(Lung_data_complete.shape)

#Removing NA's and nan's from the dataframe 
Lung_data_red = Lung_data_complete.dropna(axis=0, how='any')
#0 observations are removed 
print(Lung_data_red.shape)

#Renaming the response variable (1 - normal, 2 - lung cancer)
Lung_data_red.rename(columns={"Sam_Tissue": "Class"}, inplace=True)


(576, 1001)
(576, 1001)


### Previous step that filters the most variable genes in terms of their variance

In [13]:
#Notes: 
#1) In the Lung_data data frame, the gene-expression column order is from the most variable to the less variable, 
#in terms of their variance. 
#2) The Lung_data contains the most 1000 expressed genes of a total of ____genes of the
# genomic dataset from ________ located in the Lung Cancer Explorer (LCE) database 
# http://lce.biohpc.swmed.edu/.

#Number of most expressed gene selected: 
gen_p = 250

#Reduce dataframe with the most expressed genes given by gen_p 
#There is one variable before the genes: Class
#To include the Class variable, we need to select--> gene_data_complete.iloc[:,0: gen_p + 1]
gene_data_original = Lung_data_red.iloc[:,0: gen_p + 1]

#From Python to R
%R -i gene_data_original


### Implementation of Stable SMOTE ( Synthetic Minority Oversampling TEchnique) 

In [15]:
np.random.seed(12345)

neighbor = 5 #Number of neighboors selected

stable_smote = stable_SMOTE(neighbor)
gene_data_original_stable_smote = stable_smote.fit_sample(gene_data_original)

#From Python to R
%R -i gene_data_original_stable_smote

Number of minority instances needed
458


In [16]:
%%R

#Design matrix X and response y
X <- gene_data_original_stable_smote %>% select(-c("Class"))
y <- gene_data_original_stable_smote %>% select(c("Class"))


#Matrix preparation
X_matrix <- as.matrix(X)
y_matrix <- as.matrix(y)

# Applying the Vineknockoffs methodology with derandomized knockoffs using e-values for FDR control

In [17]:
%%R
# A list needed for the vineknockoffs sampling procedure
ls_Xk_vineknockoffs <- list()

In [18]:
#From R to pyhton
%R -o X

vine_ko = VineKnockoffs()
vine_ko.fit_vine_copula_knockoffs(x_train=X.to_numpy())  

/home/romanv/anaconda3/lib/python3.9/site-packages/vineknockoffs/copulas.py:66: UserWarning: 1. parameter of GaussianCopula copula adjusted from 0.9990685404557398 to 0.9989
  warnings.warn(f'{i_par+1}. parameter of {self.__class__.__name__} copula adjusted '
/home/romanv/anaconda3/lib/python3.9/site-packages/vineknockoffs/copulas.py:66: UserWarning: 1. parameter of GaussianCopula copula adjusted from 0.9991871210522114 to 0.9989
  warnings.warn(f'{i_par+1}. parameter of {self.__class__.__name__} copula adjusted '
/home/romanv/anaconda3/lib/python3.9/site-packages/vineknockoffs/copulas.py:66: UserWarning: 1. parameter of GaussianCopula copula adjusted from 0.9993115439436372 to 0.9989
  warnings.warn(f'{i_par+1}. parameter of {self.__class__.__name__} copula adjusted '
/home/romanv/anaconda3/lib/python3.9/site-packages/vineknockoffs/copulas.py:66: UserWarning: 1. parameter of GaussianCopula copula adjusted from 0.9991140545095449 to 0.9989
  warnings.warn(f'{i_par+1}. parameter of {sel

In [19]:
np_X = X.to_numpy()
np_errors = np.zeros(M)


ls_X_M = [np_X]*M
ls_vine_ko_M = [vine_ko]*M
ls_m = list(range(M))

#Parallel code with Joblib
ls_Xk_vineknockoffs_py = Parallel(n_jobs=jobs)(delayed(vineknockoffs_generation)(X,vine_ko, m) for X, vine_ko, m  in zip(ls_X_M, ls_vine_ko_M, ls_m))
  
#Error filtration
ls_Xk_vineknockoffs_py_red = [x for x in ls_Xk_vineknockoffs_py if x is not None]
M_red = len(ls_Xk_vineknockoffs_py_red)
%R -i M_red 
    
np_errors = M - M_red
print("Errors: ")
print(np_errors)

#Assignation of the vineknockoffs
for m in range(M_red):
    Xk_vineknockoffs_py = ls_Xk_vineknockoffs_py_red[m]
    %R -i Xk_vineknockoffs_py
    %R -i m
    %R ls_Xk_vineknockoffs[[m+1]] <- Xk_vineknockoffs_py
  
if np_errors==M:
  print("vineknockoffs selection error")  


    

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4

R[write to console]: Loaded glmnet 4.1-4



Errors: 
28


In [20]:
%%R

#Target FDR
alpha_vineknockoffs <- 0.2

#Aplication of the derandomized procedure using e-values
time_ekn_vineknockoffs <- system.time(
res_vineknockoffs <- ekn_vineknockoffs(X=X_matrix, y=y_matrix, ls_Xk_vineknockoffs, M_red, M_lasso, alpha=alpha_vineknockoffs, gamma=alpha_vineknockoffs/2, lasso_family, n_cores)
)
print("Time for running the e-knockoff procedure:")
print(time_ekn_vineknockoffs)

#Vector of integers that indicates the selected non-nulls position 
rej_vineknockoffs <- sort(res_vineknockoffs$rej)
print(rej_vineknockoffs)

#Showing the variables selected (coefficients different to zero)
print("Gene selected by the vineknockoffs procedure: ")
print(colnames(X)[rej_vineknockoffs])
print(paste0("The number of selected variables is: ",length(rej_vineknockoffs)) )


[1] "Time for running the e-knockoff procedure:"
   user  system elapsed 
 19.342   1.675  68.297 
 [1]  49  76  77 164 168 197 206 211 214 219 228
[1] "Gene selected by the vineknockoffs procedure: "
 [1] "112937" "84740"  "1591"   "142683" "26154"  "91683"  "84985"  "22977" 
 [9] "3359"   "338707" "732"   
[1] "The number of selected variables is: 11"


# Results of selected features

###  Selected variables by the vineknockoff procedure -->FDR=0.2

In [21]:
%%R
#Showing the variables selected (coefficients different to zero)
print("Gene selected by the vineknockoffs procedure: ")
print(colnames(X)[rej_vineknockoffs])
print(paste0("The number of selected variables is: ",length(rej_vineknockoffs)) )

[1] "Gene selected by the vineknockoffs procedure: "
 [1] "112937" "84740"  "1591"   "142683" "26154"  "91683"  "84985"  "22977" 
 [9] "3359"   "338707" "732"   
[1] "The number of selected variables is: 11"


In [22]:
tf = timer()
print('Time (min) taken to run all is:',round((tf-ti)/60,4))

Time (min) taken to run all is: 30366.8155
